### SETUP

In [ ]:
!pip install --user tfx tensorflow Pillow tensorflow_datasets matplotlib azure-storage-blob

### Imports


In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import IPython.display as display
from azure.storage.blob import BlobServiceClient, AccountSasPermissions, ResourceTypes
from datetime import datetime, timedelta
import random
from io import BytesIO

### Downloading images from Azure

In [ ]:
connection_string = "DefaultEndpointsProtocol=https;AccountName=datacentricthesis;AccountKey=Z2yIApz/GjhHPu28cAclxOnaqChRERQlrmGkfqfDcpCLlBRo/oPBN8n3Mhg6cRVFR5b/iS0ZxZ/D+ASt378Qfw==;EndpointSuffix=core.windows.net"
account_name = "datacentricthesis"

In [ ]:
def download_directory_from_blob_storage(blob_service_client, container_name, destination_directory, prefix=""):
  container_client = blob_service_client.get_container_client(container_name)
  blob_list = container_client.list_blobs(name_starts_with=prefix)
  
  for blob in blob_list:
    blob_path = os.path.relpath(blob.name, prefix)
    local_path = os.path.join(destination_directory, blob_path)

    os.makedirs(os.path.dirname(local_path), exist_ok=True)

    blob_client = container_client.get_blob_client(blob.name)
    with open(local_path, "wb") as file:
        file.write(blob_client.download_blob().readall())

In [ ]:
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_name = "flowers"
destination_directory = "./azure_blob_storage_test"
prefix = "flower_photos/"

if not (os.path.exists(destination_directory)):
  print("Only create and download if destination doesn't extist so we don't override it \n")
  
  download_directory_from_blob_storage(blob_service_client, container_name, destination_directory, prefix)

### Manually splitting dataset into train and test directories

In [ ]:
import os
import shutil
import random

def split_dataset_into_train_and_test(image_directory, output_directory, train_split=0.8):
    # List all the category subdirectories in the main directory
    categories = [d for d in os.listdir(image_directory) if os.path.isdir(os.path.join(image_directory, d))]

    # Create train and test directories in the output directory
    train_directory = os.path.join(output_directory, 'train')
    test_directory = os.path.join(output_directory, 'test')

    os.makedirs(train_directory, exist_ok=True)
    os.makedirs(test_directory, exist_ok=True)

    # Iterate through each category subdirectory
    for category in categories:
        category_directory = os.path.join(image_directory, category)

        # List all the image files in the category subdirectory
        images = os.listdir(category_directory)
        images = [img for img in images if img.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.gif'))]

        # Shuffle the list of image files
        random.shuffle(images)

        # Create category subdirectories in train and test directories
        train_category_directory = os.path.join(train_directory, category)
        test_category_directory = os.path.join(test_directory, category)

        os.makedirs(train_category_directory, exist_ok=True)
        os.makedirs(test_category_directory, exist_ok=True)

        # Split the image files into train and test sets
        split_index = int(len(images) * train_split)
        train_images = images[:split_index]
        test_images = images[split_index:]

        # Copy the images to the respective directories
        for img in train_images:
            shutil.copy2(os.path.join(category_directory, img), os.path.join(train_category_directory, img))

        for img in test_images:
            shutil.copy2(os.path.join(category_directory, img), os.path.join(test_category_directory, img))

        print(f'Successfully split {len(images)} images in category "{category}" into {len(train_images)} train and {len(test_images)} test images.')

image_directory = 'azure_blob_storage_test'
data_root ='flowers'
os.makedirs(data_root, exist_ok=True)
split_dataset_into_train_and_test(image_directory, data_root)


### Darken images in train part of dataset



In [ ]:
!pip install cleanvison

#### Report amount of dark images in set - Try 1

In [ ]:
# Show how many issues with dataset
# Finding faults with train images.
from cleanvision.imagelab import Imagelab

# Specify path to folder containing the image files in your dataset
imagelab = Imagelab(data_path="flowers/train/")

issue_types = {"dark": {}}
imagelab.find_issues(issue_types=issue_types)

# Produce a report with only the specified issue_types
imagelab.report()



In [ ]:
# Run the darken image script to darken X% of images and place into a new dir
%run 'scripts/darken_image_script.py'

#### Report amount of dark images in set - Try 2

In [ ]:
# Show how many issues with dataset
# Finding faults with train images.

# Specify path to folder containing the image files in your dataset
imagelab = Imagelab(data_path="darkened_images/")

issue_types = {"dark": {}}
imagelab.find_issues(issue_types=issue_types)

# Produce a report with only the specified issue_types
imagelab.report()


Number of darked images should be around the same amount specified in script, that would be 25% or around 700 images for us

### Create a new dir with dark images train/test split

In [ ]:
from scripts import create_train_dataset_from_images as ctd

ctd.create_train_dataset_from_images('darkened_images', 'flowers_original_dark_dataset','train')
ctd.create_train_dataset_from_images('flowers/test', 'flowers_original_dark_dataset','test')

file_count = 0
for root, dirs, files in os.walk('flowers_original_dark_dataset/train'):
    file_count += len(files)

print(f"The total number of files in flowers_original_dark_dataset/train and its subdirectories is: {file_count}")

### Run the script for removing the dark images from the dataset

In [ ]:
%run 'scripts/remove_dark_images.py'

In [ ]:
from scripts import create_train_dataset_from_images as ctd

ctd.create_train_dataset_from_images('darkened_images', 'flowers_improved_dataset','train')
ctd.create_train_dataset_from_images('flowers/test', 'flowers_improved_dataset','test')

file_count_original_dark_images = 0
for root, dirs, files in os.walk('flowers_improved_dataset/train'):
    file_count_original_dark_images += len(files)

print(f"The total number of files in flowers_improved_dataset/train and its subdirectories is: {file_count_original_dark_images}")

## Setting up variables

In [ ]:

PIPELINE_NAME = 'flower_pipeline'
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
ORIGINAL_DARK_TF_RECORD_ROOT = 'flowers_original_dark_dataset-tfrecord'
IMPROVED_TF_RECORD_ROOT = 'flowers_improved_dataset-tfrecord'

#data_root ='flowers'
ORIGINAL_DARK_DATA_ROOT = 'flowers_original_dark_dataset'
IMPROVED_DARK_DATA_ROOT = 'flowers_improved_dataset'


SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'flowers.db')


#IMG_HEIGHT = 180
#IMG_WIDTH = 180





### Convert original dark images to tf.record files

In [ ]:
from scripts import convert_img_to_tfrecord_script as citfs


train_data_dir = os.path.join(ORIGINAL_DARK_DATA_ROOT, 'train')
validation_dir = os.path.join(ORIGINAL_DARK_DATA_ROOT, 'test')

os.makedirs(ORIGINAL_DARK_TF_RECORD_ROOT + '/train', exist_ok=True)
os.makedirs(ORIGINAL_DARK_TF_RECORD_ROOT + '/test', exist_ok=True)

train_tfrecord = ORIGINAL_DARK_TF_RECORD_ROOT + '/train/train_data.tfrecords'
test_tfrecord = ORIGINAL_DARK_TF_RECORD_ROOT + '/test/test_data.tfrecords'


citfs.open_and_write_TFRercord(train_data_dir, validation_dir, train_tfrecord, test_tfrecord)

### Convert improved images to tf.record files

In [ ]:
from scripts import convert_img_to_tfrecord_script as citfs


train_data_dir = os.path.join(IMPROVED_DARK_DATA_ROOT, 'train')
validation_dir = os.path.join(IMPROVED_DARK_DATA_ROOT, 'test')

os.makedirs(IMPROVED_TF_RECORD_ROOT + '/train', exist_ok=True)
os.makedirs(IMPROVED_TF_RECORD_ROOT + '/test', exist_ok=True)

train_tfrecord = IMPROVED_TF_RECORD_ROOT + '/train/train_data.tfrecords'
test_tfrecord = IMPROVED_TF_RECORD_ROOT + '/test/test_data.tfrecords'


citfs.open_and_write_TFRercord(train_data_dir, validation_dir, train_tfrecord, test_tfrecord)

### Checking if decoding tfrecord file works

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

def _parse_function(example_proto):
    feature_description = {
        'label': tf.io.FixedLenFeature([], tf.int64),
        'image_raw': tf.io.FixedLenFeature([], tf.string),
    }
    parsed_features = tf.io.parse_single_example(example_proto, feature_description)
    return parsed_features

def display_first_matching_by_label(tfrecord_file, label_number):
    label_to_flower = {
        0: 'daisy',
        1: 'dandelion',
        2: 'roses',
        3: 'sunflowers',
        4: 'tulips'
    }
    
    raw_image_dataset = tf.data.TFRecordDataset(tfrecord_file)
    image_dataset = raw_image_dataset.map(_parse_function)

    for image_features in image_dataset:
        label = image_features['label']
        if label == label_number:
            encoded_image = image_features['image_raw'].numpy()
            decoded_image = tf.image.decode_jpeg(encoded_image, channels=3)
            plt.imshow(decoded_image)
            plt.title(f'Label: {label} {label_to_flower[label_number]}')
            plt.show()
            break

# Display the first image with the specified label (0 to 4) from the train TFRecord file
display_first_matching_by_label(train_tfrecord, 2)


### Model code

In [ ]:
_trainer_module_file = 'flower_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

import tensorflow as tf
from tfx import v1 as tfx
import matplotlib.pyplot as plt
import tensorflow_transform as tft
from tfx_bsl.tfxio import dataset_options
import os
import gzip
from typing import List
from tensorflow_metadata.proto.v0 import schema_pb2
from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_transform.tf_metadata import schema_utils

_IMAGE_KEY = 'image_raw'
_LABEL_KEY = 'label'
IMG_HEIGHT = 180
IMG_WIDTH = 180
batch_size = 6

# TODO: Change this to what the actual train/test dir amount is, can be found with a print somewhere above
#_TRAIN_DATA_SIZE = 2934
_TRAIN_DATA_SIZE = 2203
_TEST_DATA_SIZE = 736


def _transformed_name(key):
  return key + '_xf'

# TFX Transform will call this function.
def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.
  Args:
    inputs: map from feature keys to raw not-yet-transformed features.
  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
  print("Transformer preprocessing is running....")
  image_features = tf.map_fn(
      lambda x: tf.io.decode_jpeg(x[0], channels=3),
      inputs[_IMAGE_KEY],
      dtype=tf.uint8)
  
  image_features = tf.image.resize(image_features, [IMG_HEIGHT, IMG_WIDTH])
  image_features = tf.cast(image_features, tf.float32)
  # Normalize the image features
  # image_features = image_features / 255.0

  image_features = tf.keras.applications.mobilenet.preprocess_input(
     image_features)
  outputs[_transformed_name(_IMAGE_KEY)] = image_features
  outputs[_transformed_name(_LABEL_KEY)] = inputs[_LABEL_KEY]
  
  return outputs


# Define a function to parse the tfrecord dataset
def _parse_function(features, label):
   image = features[_transformed_name(_IMAGE_KEY)]
   return image, label

def _get_serve_image_fn(model):
  """Returns a function that feeds the input tensor into the model."""
  print("_get_serve_image_fn is running...\n\n\n")
  @tf.function
  def serve_image_fn(image_tensor):
      
    print("serve_image_fn is running...\n\n\n")
    """Returns the output to be used in the serving signature.
    Args:
      image_tensor: A tensor represeting input image. The image should have 3
        channels.
    Returns:
      The model's predicton on input image tensor
    """
    return model(image_tensor)

  return serve_image_fn

def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 12) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """  
  dataset = data_accessor.tf_dataset_factory(
      file_pattern,
      dataset_options.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_transformed_name(_LABEL_KEY)),
      tf_transform_output.transformed_metadata.schema)
  
  return dataset

# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
    
    print("\n\n\n Trainer component is running ... \n\n\n")

    epochs = 4
    # attempt at making a different model architecture that should be smaller than the original
    
    """
    OLD MODEL
    model = tf.keras.Sequential([
      #tf.keras.layers.Rescaling(1./255),
      tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),

      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(1, activation='sigmoid'),
      tf.keras.layers.Dense(5) # 5 for num of classes of flowers
    ])
    """
    model = tf.keras.Sequential([
      tf.keras.layers.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), name=_transformed_name(_IMAGE_KEY)),
      tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(5) # 5 for num of classes of flowers
    ])

 
    # Compile the model
    # OLD CODE
    #model.compile(optimizer='adam',
    #          loss=tf.keras.losses.BinaryCrossentropy(),
    #          metrics=['accuracy'])
    # NEW CODE
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    # Print a summary of the model
    model.summary()

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
 
    train_ds = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      tf_transform_output,
      batch_size=batch_size)
    
    train_ds = train_ds.map(_parse_function)

    eval_ds = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      tf_transform_output,
      batch_size=batch_size)
    
    eval_ds = eval_ds.map(_parse_function)

    steps_per_epoch = int(_TRAIN_DATA_SIZE / batch_size)
    validation_steps = int(_TEST_DATA_SIZE / batch_size)

    # Train our model
    history = model.fit(train_ds,
                        epochs=epochs,
                        validation_data=eval_ds, 
                        steps_per_epoch=steps_per_epoch, 
                        validation_steps=validation_steps)

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(epochs)

    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()


    # Prepare the TFLite model used for serving in MLKit
    signatures = {
        'serving_default':
            _get_serve_image_fn(model).get_concrete_function(
                tf.TensorSpec(
                    shape=[None, IMG_HEIGHT, IMG_WIDTH, 3],
                    dtype=tf.float32,
                    name=_transformed_name(_IMAGE_KEY)))
    }
    print(signatures)
    # The result of the training should be saved in `fn_args.serving_model_dir`
    # directory.
    model.save(fn_args.serving_model_dir, save_format='tf')

### Pipeline definition

In [ ]:

### This pipeline hard codes the dataset path inside the pipeline instead of as a input when running orchestrator
import tensorflow_model_analysis as tfma

from tfx.proto import example_gen_pb2
from tfx.components import ImportExampleGen, Evaluator
from tfx import v1 as tfx
from tfx.dsl.components.common import resolver
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.proto import example_gen_pb2
from tfx.types.standard_artifacts import Model

import tensorflow_model_analysis as tfma

from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy

def _create_pipeline(pipeline_name: str, pipeline_root: str, 
                     tf_record_path: str, module_file: str, 
                     serving_model_dir: str, 
                     metadata_path: str,   accuracy_threshold: float = 0.15) -> tfx.dsl.Pipeline:
  print("Pipeline creation is running...\n\n\n")
 
  input_config = example_gen_pb2.Input(splits=[
      example_gen_pb2.Input.Split(name='train', pattern='train/*'),
      example_gen_pb2.Input.Split(name='eval', pattern='test/*')
  ])
#TODO: Move hardcoded input base into an external variable and send it into the pipeline
  # Brings data into the pipeline.
  example_gen = ImportExampleGen(
      input_base=tf_record_path, input_config=input_config)

  # Computes statistics over data for visualization and schema generation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])
  
  # Generates schema based on the generated statistics.
  schema_gen = tfx.components.SchemaGen(
      statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)

  #Performs anomaly detection based on statistics and data schema.
  example_validator = tfx.components.ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema = schema_gen.outputs['schema']
  )

  # Performs transformations and feature engineering in training and serving.
  transform = tfx.components.Transform(
      examples=example_gen.outputs['examples'],
      schema=schema_gen.outputs['schema'],
      module_file=module_file)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=transform.outputs['transformed_examples'],
      transform_graph=transform.outputs['transform_graph'],
      schema= schema_gen.outputs['schema'],
      )
  
  
  # Get the latest blessed model for model validation.
  model_resolver = resolver.Resolver(
      #strategy_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
      strategy_class=LatestBlessedModelStrategy,
      model=Channel(type=Model),
      model_blessing=Channel(
          type=ModelBlessing)).with_id('latest_blessed_model_resolver')
  
  
  # Uses TFMA to compute evaluation statistics over features of a model and
  # perform quality validation of a candidate model (compare to a baseline).
  eval_config = tfma.EvalConfig(
      model_specs=[tfma.ModelSpec(label_key='label_xf')],
      slicing_specs=[tfma.SlicingSpec()],
      metrics_specs=[
          tfma.MetricsSpec(metrics=[
              tfma.MetricConfig(
                  class_name='SparseCategoricalAccuracy',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': accuracy_threshold}),
                      # Change threshold will be ignored if there is no
                      # baseline model resolved from MLMD (first run).
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                          absolute={'value': -1e-3})))
          ])
      ])

  # Uses TFMA to compute the evaluation statistics over features of a model.
  # We evaluate using the materialized examples that are output by Transform
  # because
  # 1. the decoding_jpeg function currently performed within Transform are not
  # compatible with TFLite.
  # 2. MLKit requires deserialized (float32) tensor image inputs
  # Note that for deployment, the same logic that is performed within Transform
  # must be reproduced client-side.
  evaluator = Evaluator(
      examples=transform.outputs['transformed_examples'],
      model=trainer.outputs['model'],
      baseline_model=model_resolver.outputs['model'],
      eval_config=eval_config)
  
  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      model_blessing=evaluator.outputs['blessing'], # Pass an evaluation result.
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))
  
  # Following three components will be included in the pipeline.
  components = [
      example_gen,
      statistics_gen,
      schema_gen,
      example_validator,
      transform,
      trainer,
      model_resolver,
      evaluator,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

### Run Pipeline First Attempt With Dark Images

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      tf_record_path=ORIGINAL_DARK_TF_RECORD_ROOT,
      module_file=_trainer_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

### Run Pipeline First Attempt WITHOUT Dark Images

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      tf_record_path=IMPROVED_TF_RECORD_ROOT,
      module_file=_trainer_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

### Getting Evaluator component and visualizing statistics

### Result querying

In [ ]:
# List files in created model directory.
!find {SERVING_MODEL_DIR}